In [1]:
%matplotlib inline

In [2]:
import os
import os.path as op
import numpy as np
from mayavi import mlab
import fnmatch
import pandas as pd
from tvtk.api import tvtk
from collections import defaultdict
from vtk_viz import vtkvizfuncs as vz
import cPickle as pickle
# pylint: disable=C0103
# pylint:disable=E1101
vtkF = defaultdict(dict)
mombud = defaultdict(dict)
datadir = op.join(os.getcwd(),'data')

# =============================================================================
# filelist and graph list
# =============================================================================
for root, dirs, files in os.walk(op.join(datadir, 'normSkel')):
    for i in files:
        if fnmatch.fnmatch(i, '*skeleton.vtk'):
            media = root.rsplit(os.sep, 1)[1]
            vtkF[media][i[5:-13]] = op.join(root, i)
for root, dirs, files in os.walk(op.join(datadir, 'csv')):
    for i in files:
        if fnmatch.fnmatch(i, 'YP*csv'):
            mombud[i[:-4]] = op.join(root, i)

filekeys = {item: vtkF[media][item] for media
            in sorted(vtkF.keys()) for item
            in sorted(vtkF[media].keys())}

DataSize = pd.read_table(op.join(datadir, 'csv', 'Results.txt'))
df = DataSize.ix[:, 1:]
df['cell'] = df.ix[:, 'Label'].apply(lambda x: x.partition(':')[2])
df['vol'] = 4 / 3 * np.pi * (df.Major * .055 / 2) * (df.Minor * .055 / 2) ** 2


In [3]:
dfmb = pd.DataFrame(columns=['base', 'neck', 'tip', 'media'])
mlab.close(all=True)
for _, key in enumerate(sorted(mombud.keys())[5:6]):
    df1 = pd.read_csv(op.join(datadir, 'csv', '%s.csv' % key),
                      header=0,
                      names=['x', 'y', 'z'],
                      index_col=0)
    tip = np.array(df1.ix['tip'])
    base = np.array(df1.ix['base'])
    neck = np.array(df1.ix['neck'])

    filekey = key
    df2 = vz.getelipspar(filekey, df)
    df2 = df2.sort_values('vol')
    df2.reset_index(drop=True, inplace=True)
    df2.index = ['bud', 'mom']
    df2['center'] = zip((df2.X - 25) * .055, (225 - df2.Y) * .055)
    figone = mlab.figure(figure=filekey,
                         size=(800, 600),
                         bgcolor=(0., 0., 0.))

    figone.scene.disable_render = True
    _, vtkobj, tubeout = vz.cellplot(figone, filekeys, filekey)
    xmin, xmax, ymin, ymax, zmin, zmax = vtkobj.outputs[0].bounds

In [5]:
xmax

7.310719966888428

In [6]:
dfmb

,base,neck,tip,media


In [7]:
df1


,x,y,z
neck,3.7284,5.5492,3.1483
base,8.5468,6.2690,3.6362
tip,2.1189,5.2158,3.0018
centerpt,2.7094,NaN,NaN


In [8]:
df2

,Label,X,Y,Major,Minor,Angle,cell,vol,center
bud,CombinedStack-1.tif:YPD_042515_011_RFPstack_012,79.146,128.285,33.542,30.785,3.498,YPD_042515_011_RFPstack_012,2.076898,"(2.97803, 5.319325)"
mom,CombinedStack-1.tif:YPD_042515_011_RFPstack_012,138.366,120.717,94.517,83.251,6.516,YPD_042515_011_RFPstack_012,42.799313,"(6.23513, 5.735565)"
